In [0]:
import numpy as np
import tensorflow as tf
import os

In [0]:
!pip install tensorboardcolab

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://32a58197.ngrok.io


## **Get and inpect data**

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = x_train.reshape((x_train.shape[0], 28*28))
x_test = x_test.reshape((x_test.shape[0], 28*28))
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [100]:
print(y_test[:1])
print('Training set', x_train.shape, y_train.shape)
print('Test set', x_test.shape, y_test.shape)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Training set (60000, 784) (60000, 10)
Test set (10000, 784) (10000, 10)


## **Define the neural network layers**

In [0]:
def hidden(data, in_dims, out_dims, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=0.01), name="W")
        #w = tf.Variable(tf.zeros([in_dims, out_dims]), name="W")
        b = tf.Variable(tf.zeros([out_dims]), name="B")
        out = tf.matmul(data, w) + b
        act = tf.nn.relu(out)
        return act

In [0]:
def out_layer(data, in_dims, out_dims, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([in_dims, out_dims], stddev=0.01), name="W")
        #w = tf.Variable(tf.zeros([in_dims, out_dims]), name="W")
        b = tf.Variable(tf.zeros(out_dims))
        out = tf.matmul(data, w) + b
        return out

## **Define the hyperparameters**

In [0]:
batch_size = 50
num_of_epochs = 4
image_size = 28
num_labels = 10
#log_dir = '/notebooks/data/demo/'
lr = 1e-2

## **give a name to the run**

In [0]:
run_name = 'run-demo-1-lr-' + str(lr) + '-b' + str(batch_size) + 'h_' + 'L-100-150-50-25'

## **Reset the graph**

In [0]:
tf.reset_default_graph()

## **Define inputs**

In [106]:
x = tf.placeholder(tf.float32, shape=[None, image_size * image_size], name="X")
y_label = tf.placeholder(tf.float32, shape=[None, num_labels], name="Y_label")

image_shaped_input = tf.reshape(x, [-1, image_size, image_size, 1])
tf.summary.image('input', image_shaped_input, 10)

<tf.Tensor 'input:0' shape=() dtype=string>

## **Define the network**

In [0]:
hidden1 = hidden(x, image_size * image_size, 100, "hidden1")
hidden2 = hidden(hidden1, 100, 150, "hidden2")
hidden3 = hidden(hidden2, 150, 50, "hidden3")
hidden4 = hidden(hidden3, 50, 25, "hidden4")
pred = out_layer(hidden4, 25, num_labels, "pred")

## **Define the operations**

In [108]:
with tf.name_scope("xent"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_label))
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
tf.summary.scalar('acc', accuracy)
tf.summary.scalar('loss', cost)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [0]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(lr).minimize(cost)

## **Start a tensorflow session**

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## **Setup the tensorboard parameters**

In [0]:
summ = tf.summary.merge_all()
train_writer = tbc.get_writer()
#train_writer = tf.summary.FileWriter(log_dir + 'train/' + run_name)
train_writer.add_graph(sess.graph)

## **Train the model**

In [112]:
total_train_indexes = len(x_train)
for epoch in range(num_of_epochs):
    i = 0
    epoch_loss = 0
    while i < total_train_indexes:
        start = i
        end = i + batch_size

        if end > total_train_indexes:
            end = total_train_indexes

        batch_dataset = x_train[start:end]
        batch_labels = y_train[start:end]

        input_dict = {x: batch_dataset, y_label: batch_labels}
        _, epoch_accuracy, loss, s = sess.run([train_step, accuracy, cost, summ], feed_dict=input_dict)
        epoch_loss += loss
        train_writer.add_summary(s, i + total_train_indexes * epoch)
        i += batch_size
    valid_accuracy, valid_loss = sess.run([accuracy, cost], 
                                                   feed_dict={x: x_test, 
                                                              y_label: y_test})  
    
    print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'train_loss:',epoch_loss, 'train_ccuracy:', epoch_accuracy)
    print('Epoch', epoch + 1, 'completed out of',num_of_epochs,'valid_loss:',valid_loss, 'valid_accuracy:', valid_accuracy)
    
test_accuracy, test_loss = sess.run([accuracy, cost], feed_dict={x: x_test, y_label: y_test})

print("Test accuracy:", test_accuracy, "Test loss:", test_loss)

Epoch 1 completed out of 4 train_loss: 893.8849964886904 train_ccuracy: 0.82
Epoch 1 completed out of 4 valid_loss: 0.5135463 valid_accuracy: 0.8109
Epoch 2 completed out of 4 train_loss: 590.7147527784109 train_ccuracy: 0.86
Epoch 2 completed out of 4 valid_loss: 0.5334833 valid_accuracy: 0.8124
Epoch 3 completed out of 4 train_loss: 546.2189397513866 train_ccuracy: 0.86
Epoch 3 completed out of 4 valid_loss: 0.5025665 valid_accuracy: 0.8259
Epoch 4 completed out of 4 train_loss: 502.4646100252867 train_ccuracy: 0.84
Epoch 4 completed out of 4 valid_loss: 0.49323618 valid_accuracy: 0.8284
Test accuracy: 0.8284 Test loss: 0.49323618
